In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import category_encoders as ce

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read data
ss, test, train = [
    pd.read_csv(
        os.path.join(dirname, filename)
    ) 
    for dirname, _, filenames in os.walk('/kaggle/input')
    for filename in filenames
]

In [ ]:
# Preprocessing
# Transform categorical features, drop unnecessary columns
y, train = train.target, train.drop(['enrollee_id', 'target'], axis=1)
encoder = ce.BinaryEncoder(cols=[
    'city', 
    'gender',
    'relevent_experience',
    'enrolled_university', 
    'education_level',
    'major_discipline', 
    'experience',
    'company_size', 
    'company_type',
    'last_new_job',
])
encoder.fit(train, y)
X = encoder.transform(train)
X_test = encoder.transform(test.drop('enrollee_id', axis=1))

In [ ]:
# Train a classifier
xg = xgb.XGBClassifier()
xg.fit(X, y)

In [ ]:
# Predict labels on test data
labels = xg.predict(X_test)
result = pd.DataFrame({'enrollee_id': test.enrollee_id, 'target': labels})

In [ ]:
result.to_csv('submission.csv', index=False)

In [ ]:
ls